# 1. Peridically fetch traffic trends and attack spikes from clodfare and/or IPS from abuse IPDP
# 2. Classify IPS with high DDos confidence score using machine learning
# 3, Remote IPs to coordinate and show them in fromtend globe

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
import requests 

API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
url="https://api.cloudflare.com/client/v4/user/tokens/verify"

headers={
    "Authorization": f"Bearer {API_TOKEN}"
}

response=requests.get(url=url,headers=headers)
# print(response.status_code)
# print(response.json())

In [12]:
API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

PARAMS = {
    "aggInterval": "1h",
    "dateRange": "1d",
    "format": "json"
}

def fetch_data(url):
    response = requests.get(url, headers=HEADERS, params=PARAMS)
    if response.status_code == 200:
        return response.json().get("result", {})
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

# Layer 3 URL
vector_url = "https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/vector"
ip_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/ip_version"
protocol_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/protocol"
bitrate_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/bitrate"
duration_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/duration"
# Fetch L3 Data
vector_data = fetch_data(vector_url)
ip_data=fetch_data(ip_url)
protocol_data=fetch_data(protocol_url)
bitrate_data=fetch_data(bitrate_url)
duration_data=fetch_data(duration_url)


# Convert to DataFrame
df_vector = pd.DataFrame(vector_data)
df_ip = pd.DataFrame(ip_data)
df_protocol = pd.DataFrame(protocol_data)
df_bitrate = pd.DataFrame(bitrate_data)
df_duration = pd.DataFrame(duration_data)
# print(df_ip)


In [60]:
df_bitrate

,Serie_0 timestamps,Serie_0 U N D E R_500_ M B P S,Serie_0 _1_ G B P S_ T O_10_ G B P S,Serie_0 _500_ M B P S_ T O_1_ G B P S,Serie_0 O V E R_100_ G B P S,Serie_0 _10_ G B P S_ T O_100_ G B P S
0,2025-08-14T10:00:00Z,81.131980,9.792011,9.000149,0.075860,0.000000
1,2025-08-14T11:00:00Z,62.488012,0.009236,37.499369,0.003383,0.000000
2,2025-08-14T12:00:00Z,90.465360,0.206702,9.205974,0.073664,0.048300
3,2025-08-14T13:00:00Z,75.494841,0.002635,24.422893,0.079631,0.000000
4,2025-08-14T14:00:00Z,57.119219,0.044188,42.824617,0.011975,0.000000
5,2025-08-14T15:00:00Z,78.105448,2.028064,19.863435,0.003052,0.000000
6,2025-08-14T16:00:00Z,98.064433,1.764390,0.163993,0.007184,0.000000
7,2025-08-14T17:00:00Z,94.276263,3.040474,2.637845,0.045418,0.000000
8,2025-08-14T18:00:00Z,42.627691,36.112559,21.254180,0.005570,0.000000
9,2025-08-14T19:00:00Z,94.674890,1.379140,3.936891,0.009079,0.000000


In [48]:
data=vector_data["serie_0"]

In [53]:
df=pd.DataFrame(data)
df_vector=df.melt(
    id_vars=["timestamps"],
    var_name="attack_type",
    value_name="value"
)

In [51]:
# for x in vector_data["serie_0"]:
#     df[x]=pd.DataFrame(vector_data["serie_0"][x])

In [57]:
df_vector

,timestamps,attack_type,value
0,2025-08-14T10:00:00Z,UDP Flood,43.50912
1,2025-08-14T11:00:00Z,UDP Flood,37.354682
2,2025-08-14T12:00:00Z,UDP Flood,22.299304
3,2025-08-14T13:00:00Z,UDP Flood,23.950651
4,2025-08-14T14:00:00Z,UDP Flood,24.515547
...,...,...,...
235,2025-08-15T05:00:00Z,NTP Amplification,0.298622
236,2025-08-15T06:00:00Z,NTP Amplification,0.099835
237,2025-08-15T07:00:00Z,NTP Amplification,0.019396
238,2025-08-15T08:00:00Z,NTP Amplification,0.000102
